In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.metrics import classification_report

data = pd.read_csv('stroke_classification.csv')

#profile = ProfileReport(data, title='DR', explorative=True)
#profile.to_file('report.html')

target = 'stroke'
x= data.drop([target, 'pat_id'], axis=1)
y = data[target]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])


nom_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('norminal', OneHotEncoder(handle_unknown='ignore'))
])

preprocesser =ColumnTransformer(transformers=[
    ('num_feature', num_transformer, [ 'age', 'avg_glucose_level', 'bmi' ]),
    ('norminal_feature', nom_transformer, ['gender'])
])

model = Pipeline(steps=[
    ('pre_processer', preprocesser),
    ('classification', SVC())
])

params= {
    'classification__C':[0,1,2],
    'classification__kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
    'classification__degree': [2,3,4]
}

grid_search = GridSearchCV(estimator=model, param_grid=params, cv= 6, verbose=2)

grid_search.fit(x_train, y_train)

print(grid_search.best_params_)

y_predict = grid_search.predict(x_test)
print(classification_report(y_test, y_predict))

KeyError: 'result'